# <a id='toc1_'></a>[Рекомендация тарифов](#toc0_)

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

### <a id='toc1_1_1_'></a>[Цель](#toc0_)

Построить модель с максимально большим значением accuracy для определение нового тарифа.

### <a id='toc1_1_2_'></a>[Данные](#toc0_)

Предоставленны данные о поведении клиентов, которые уже перешли на эти тарифы.

### <a id='toc1_1_3_'></a>[План](#toc0_)

<a id='toc0_'></a>    
- [Рекомендация тарифов](#toc1_)    
    - [Цель](#toc1_1_1_)    
    - [Данные](#toc1_1_2_)    
    - [План](#toc1_1_3_)    
  - [Откройте и изучите файл](#toc1_2_)    
  - [Разбейте данные на выборки](#toc1_3_)    
  - [Исследуйте модели](#toc1_4_)    
    - [Решающее дерево](#toc1_4_1_)    
    - [Случайный лес](#toc1_4_2_)    
    - [Логистическая регрессия](#toc1_4_3_)    
  - [Проверьте модель на тестовой выборке](#toc1_5_)    
  - [Вывод](#toc1_6_)    
  - [(бонус) Проверьте модели на адекватность](#toc1_7_)    
  - [Чек-лист готовности проекта](#toc1_8_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

In [11]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression 
from sklearn.metrics import accuracy_score


## <a id='toc1_2_'></a>[Откройте и изучите файл](#toc0_)

In [12]:
try:
    df = pd.read_csv('datasets/users_behavior.csv')
except:
    df = pd.read_csv('/datasets/users_behavior.csv')

display(df.head())
print(df.shape)
df.info()
print(df.duplicated().sum())


,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


(3214, 5)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB
0


In [15]:
from fast_ml.model_development import train_valid_test_split
x_train, x_valid, x_test, y_train, y_valid, y_test = train_valid_test_split(df, target='is_ultra', train_size=0.6, valid_size=0.2, test_size=0.2)

Предобработка не труебтся

## <a id='toc1_3_'></a>[Разбейте данные на выборки](#toc0_)

In [3]:
features = df.drop('is_ultra', axis=1)
target = df['is_ultra']

Так как тестовой выбор у нас нет, делим в пропорции 3:1:1

In [4]:
features_train, features_vt, target_train, target_vt = train_test_split(
    features, target, test_size=0.4, random_state=12345)
features_valid, features_test, target_valid, target_test = train_test_split(
    features_vt, target_vt, test_size=0.5, random_state=12345)   
print(features_train.shape)
print(features_valid.shape)
print(features_test.shape)

(1928, 4)
(643, 4)
(643, 4)


## <a id='toc1_4_'></a>[Исследуйте модели](#toc0_)

В теории нам сказали, что random forest обладает наилучшей точностью, но в академических целях, проверим точность всех вариантов моделей

### <a id='toc1_4_1_'></a>[Решающее дерево](#toc0_)

In [5]:
best_tree_model = None
best_result = 0
best_depth = 0
for depth in range(1, 6):
	model = DecisionTreeClassifier(random_state=12345, max_depth=depth) 
	model.fit(features_train, target_train) 
	predictions = model.predict(features_valid) 
	result = accuracy_score(target_valid, predictions) 
	if result > best_result:
		best_tree_model = model
		best_result = result
		best_depth = depth

print("Accuracy лучшей модели дерева:", best_result, 'Глубина дерева:', best_depth)

Accuracy лучшей модели дерева: 0.7853810264385692 Глубина дерева: 3


### <a id='toc1_4_2_'></a>[Случайный лес](#toc0_)

In [6]:
best_rf_model = None
best_result = 0
best_est = 0
best_depth = 0
for est in range(1, 11):
    for depth in range(1, 6):
        model = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth) 
        model.fit(features_train, target_train) 
        predictions = model.predict(features_valid) 
        result = accuracy_score(target_valid, predictions) 
        if result > best_result:
            best_rf_model = model 
            best_result = result
            best_est = est
            best_depth = depth

print('Accuracy наилучшей модели случайный лес:', best_result)
print('Количество деревьев в лесу:', best_est)
print('Глубина дерева:', best_depth)

Accuracy наилучшей модели случайный лес: 0.7931570762052877
Количество деревьев в лесу: 8
Глубина дерева: 5


### <a id='toc1_4_3_'></a>[Логистическая регрессия](#toc0_)

In [16]:
model_lr = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=1000) 
model_lr.fit(features_train, target_train)
predictions = model_lr.predict(features_valid) 
result = accuracy_score(target_valid, predictions) 
print("Accuracy модели логистической регресси:", result)

Accuracy модели логистической регресси: 0.7107309486780715


## <a id='toc1_5_'></a>[Проверьте модель на тестовой выборке](#toc0_)

In [15]:
print("Accuracy на тестовых данных")
print('Решающее дерево:', accuracy_score(target_test, best_tree_model.predict(features_test)))
print('Случайный лес:', accuracy_score(target_test, best_rf_model.predict(features_test)))
print('Логистическая регрессия:', accuracy_score(target_test, model_lr.predict(features_test)))


Accuracy на тестовых данных
Решающее дерево: 0.7791601866251944
Случайный лес: 0.7962674961119751
Логистическая регрессия: 0.6842923794712286


## <a id='toc1_6_'></a>[Вывод](#toc0_)
На тестовых данных лучшшую accuracy показала модель Random Forest с гиперпараметром `n_estimators` равным 8 и `max_depth` = 5. Accuracy ~0.80, что выше 0,75, поэтому эту модель можно брать в работу.

## <a id='toc1_7_'></a>[(бонус) Проверьте модели на адекватность](#toc0_)

Чтобы проверить модель на адекватность, сравним её Accuracy c Accuracy случайной выборки

In [9]:
target_random = pd.Series(np.random.randint(0,1), index=target_test.index)
print('Accuracy случайного выбора', accuracy_score(target_test, target_random))

Accuracy случайного выбора 0.6842923794712286


Вывод: Выбранные нами модель прошла проверку на адекватность.

## <a id='toc1_8_'></a>[Чек-лист готовности проекта](#toc0_)

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x] Jupyter Notebook открыт
- [x] Весь код исполняется без ошибок
- [x] Ячейки с кодом расположены в порядке исполнения
- [x] Выполнено задание 1: данные загружены и изучены
- [x] Выполнено задание 2: данные разбиты на три выборки
- [x] Выполнено задание 3: проведено исследование моделей
    - [x] Рассмотрено больше одной модели
    - [x] Рассмотрено хотя бы 3 значения гипепараметров для какой-нибудь модели
    - [x] Написаны выводы по результатам исследования
- [x] Выполнено задание 3: Проведено тестирование
- [x] Удалось достичь accuracy не меньше 0.75
